<a href="https://colab.research.google.com/github/TaiseiYamana/optuna_study/blob/main/optuna_tutorial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna チュートリアル3

## 参考本
https://www.ohmsha.co.jp/book/9784274230103/
## GitHub
https://github.com/pfnet-research/optuna-book

In [1]:
!pip3 install optuna

# 分散並列最適化


大規模な機械学習モデルを学習する場合は、1回のトライアルにかかる時間が長くなります。  
そのような時はハイパーパラメータの最適化を並列化したい場合があります。

Optunaを用いると、分散並列ハイパーパラメータ最適化を簡単に行う事ができます。   
Optunaの一般の分散並列最適化はRDBを用いるため、GoogleColabでは簡易的にできません。  
実行可能な環境で以下手順を実施してください。  
https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/004_distributed.html

## Optunaの分散並列最適化の仕組み

全ての最適化履歴は、MySQLやPostgreSQLなどのRDB等の共有のストレージに保存されます。  
そして、実際に最適化を実行する複数のワーカーは、それぞれが独立にこのストレージにアクセスします。
そのため任意のワーカーを追加可能で中断ができます。またワーカー間の競合は、Optunaが管理をしてくれるのでユーザは気を遣う必要がありません。


## google colabでgoogle driveをストレージに利用


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

directory_path = './drive/MyDrive/optuna_colab_db/'
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Optunaの機械学習サンプルコード
ソースコード：https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py  

FashionMNISTのクラス分類
モデルはMLPを使用

探索パラメータ:
- モデルのレイヤー数 1~3
- 各層のユニット数 4~128
- 各層のドロップアウト率 0.2~0.5
- 学習率 1e-5 ~ 1e-1
- Optimizer [Adam, RMSprop, SGD]

実行トライアル数 500
枝切り MedianPruner

In [4]:
import optuna
from optuna.trial import TrialState
from optuna.study import MaxTrialsCallback
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms
from optuna.pruners import NopPruner

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 50
N_VALID_EXAMPLES = BATCHSIZE * 10

def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        #trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        #if trial.should_prune():
        #    raise optuna.exceptions.TrialPruned()

    return accuracy

In [7]:
n_trials = 500
strage_name = "optuna_strage.sql"
study_name = 'example-study'
storage='sqlite:///' + directory_path + strage_name
print(f"storage: {storage}")
study = optuna.create_study(
    study_name = study_name,
    storage=storage,
    load_if_exists=True,
    pruner = NopPruner,
    direction='maximize',
    )

study.optimize(objective,
               n_trials=n_trials,
               callbacks=[MaxTrialsCallback(n_trials)],
               )


[I 2023-07-04 16:54:53,397] Using an existing study with name 'example-study' instead of creating a new one.


storage: sqlite:///./drive/MyDrive/optuna_colab_db/optuna_strage.sql


[I 2023-07-04 16:55:04,354] Trial 57 finished with value: 0.7671875 and parameters: {'dropout_l0': 0.476075556798528, 'dropout_l1': 0.2565067073881019, 'lr': 0.0036976606754118065, 'n_layers': 2, 'n_units_l0': 91, 'n_units_l1': 48, 'optimizer': 'RMSprop'}. Best is trial 35 with value: 0.85546875.
[I 2023-07-04 16:55:14,883] Trial 58 finished with value: 0.82734375 and parameters: {'dropout_l0': 0.49879490330069354, 'lr': 0.0021980287826029853, 'n_layers': 1, 'n_units_l0': 118, 'optimizer': 'RMSprop'}. Best is trial 35 with value: 0.85546875.
[I 2023-07-04 16:55:25,886] Trial 59 finished with value: 0.79921875 and parameters: {'dropout_l0': 0.4490609611537024, 'lr': 0.0013904252600596162, 'n_layers': 1, 'n_units_l0': 32, 'optimizer': 'RMSprop'}. Best is trial 35 with value: 0.85546875.
[I 2023-07-04 16:55:36,903] Trial 60 finished with value: 0.8328125 and parameters: {'dropout_l0': 0.46504333090536976, 'dropout_l1': 0.31775265297045374, 'lr': 0.002062434296053446, 'n_layers': 2, 'n_uni

In [8]:
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

Best objective value: 0.86484375
Best parameter: {'dropout_l0': 0.36390876509297526, 'lr': 0.004418871241308287, 'n_layers': 1, 'n_units_l0': 121, 'optimizer': 'Adam'}
